# 🌐 Strands + AgentCore Browser 통합 

## 개요

이 튜토리얼에서는 지능형 웹 자동화 및 금융 분석을 위해 Strands agent를 Amazon Bedrock AgentCore Browser tool과 함께 사용하는 방법을 배웁니다. 공식 `strands_tools.browser.AgentCoreBrowser` 통합을 사용하여 웹사이트를 탐색하고, 금융 데이터를 추출하며, AI 기반 인사이트를 제공할 수 있는 Strands agent를 생성하는 방법을 시연합니다.

### 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                        |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | Conversational                                                                   |
| Agent 유형          | Single                                                                           |
| Agentic Framework   | Strands                                                                          |
| LLM model           | Claude 3 Haiku                                                                   |
| 튜토리얼 구성 요소  | Browser automation, Financial analysis, Error recovery                           |
| 튜토리얼 분야       | Financial Technology                                                             |
| 예제 복잡도         | Intermediate                                                                     |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK, Strands Agents, Strands Agent Tools         |

### 튜토리얼 아키텍처

이 튜토리얼에서는 지능형 금융 웹사이트 분석을 위해 Strands agent를 browser tool과 함께 사용하는 방법을 설명합니다. 우리의 구현은 Strands와 AWS Bedrock AgentCore 간의 원활한 통합을 제공하는 공식 `strands_tools.browser.AgentCoreBrowser`를 사용합니다.

예제에서는 Strands agent에 자연어 지시를 보내 MarketWatch와 같은 웹사이트에서 금융 분석 작업을 수행하고, 주식 데이터와 시장 지표를 추출하며, AI 기반 인사이트를 제공합니다.

### 튜토리얼 주요 기능

* Strands agent와 함께 headless 방식으로 browser tool 사용
* strands_tools.browser를 사용한 AgentCoreBrowser 통합
* 컨텍스트 인식 AI 응답을 통한 지능형 금융 웹사이트 분석
* 폴백 전략을 사용한 오류 처리 및 응답 파싱
* 안정적인 agent 구현을 위한 구조화된 invoke 패턴
* 빠르고 효율적인 분석을 위한 Claude Haiku model

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* 구성된 AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* Strands agents 및 strands-agents-tools 패키지
* Amazon Bedrock의 Claude 3 Haiku model 액세스 권한

**구현**: 지능형 금융 분석과 함께 원활한 브라우저 자동화를 위해 공식 `strands_tools.browser.AgentCoreBrowser`를 사용합니다.

## 🎯 이 튜토리얼의 기능
- **AgentCoreBrowser 통합** `strands_tools.browser` 사용
- **MarketWatch 금융 분석** 컨텍스트 인식 AI 응답 포함
- **구조화된 Invoke 패턴** 구현용
- **Claude Haiku Model** 빠르고 효율적인 분석

## 🏗️ 아키텍처

```mermaid
graph TD
    A[👤 User Request] --> B[📋 Invoke Function]
    B --> C[🤖 Strands Agent]
    C --> D[🔧 AgentCoreBrowser Tool]
    D --> E[🌐 AWS Bedrock AgentCore]
    E --> F[🎭 Browser Automation]
    F --> G[📊 MarketWatch.com]
    G --> H[📈 Stock Data Extraction]
    H --> I[🧠 Claude Analysis]
    I --> J[📋 Financial Insights]
    
    style A fill:#1a1a2e
    style C fill:#16213e
    style D fill:#0f3460
    style E fill:#e94560
    style I fill:#f39c12
```


## 📦 Step 1: 종속성 설치

In [ ]:
# strands-agents-tools 포함하여 필요한 패키지 설치 (AgentCoreBrowser 사용을 위함)
!pip install --force-reinstall -U -r requirements.txt 

print("✅ Dependencies installed successfully!")

## 📚 Step 2: 라이브러리 임포트

In [ ]:
from strands import Agent  # Strands agent framework
from strands_tools.browser import AgentCoreBrowser  # AWS Bedrock AgentCore 브라우저 통합
import time

print("✅ Libraries imported successfully!")

## 🤖 Step 3: 최적화된 설정으로 Agent 생성

In [ ]:
def create_agent():
    """Create and configure the Strands agent with AgentCoreBrowser"""
    # AgentCoreBrowser tool 초기화 (us-west-2 리전 사용)
    agent_core_browser = AgentCoreBrowser(region="us-west-2")
    
    # Claude Haiku model과 최적화된 설정으로 agent 생성
    agent = Agent(
        tools=[agent_core_browser.browser],  # 브라우저 tool 등록
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # 빠른 분석을 위한 Haiku model
        system_prompt="""You are an intelligent financial analyst that specializes in analyzing stock and financial websites. When asked to analyze a financial website:

1. Use the browser tool to visit and interact with the website EFFICIENTLY
2. Focus on extracting key financial information QUICKLY:

**For Financial/Stock Websites (MarketWatch, Bloomberg, etc.):**
- Current stock prices and market data
- Price movements and trends (daily, weekly, monthly changes)
- Key financial metrics and ratios (P/E, Market Cap, etc.)
- Trading volume and market activity
- Recent news and market sentiment
- Analyst recommendations and price targets
- Company fundamentals and performance indicators

IMPORTANT: Work efficiently and complete your analysis within 2-3 browser interactions. Always provide specific, actionable financial insights with actual numbers and data points. Be concise but comprehensive in your analysis, focusing on the most important financial metrics for investors."""
    )
    return agent

# Agent를 전역으로 초기화
strands_agent = create_agent()

print("✅ Strands agent created with AgentCoreBrowser integration!")

## 🔧 Step 4: Invoke 함수 생성

In [ ]:
def invoke(payload):
    """Structured invoke function for agent interaction"""
    user_message = payload.get("prompt", "")
    
    try:
        print("🚀 Starting analysis...")
        start_time = time.time()
        
        # Agent 실행
        response = strands_agent(user_message)
        
        elapsed_time = time.time() - start_time
        print(f"✅ Analysis completed in {elapsed_time:.2f} seconds")
        
        # 응답에서 텍스트 추출 (content 배열의 첫 번째 요소)
        return response.message["content"][0]["text"] if response.message.get("content") else str(response)
        
    except Exception as e:
        print(f"❌ Error during analysis: {str(e)}")
        return f"Error occurred: {str(e)}"

print("✅ Invoke function created!")

# 📝 Note: 느린 웹사이트 접근 시 timeout이 발생할 수 있으므로
# 필요한 경우 timeout 처리를 구현해야 합니다.

## 📊 Step 5: MarketWatch 금융 분석 테스트

In [ ]:
# MarketWatch Tesla 페이지로 테스트
financial_test = {
    "prompt": "Please analyze the Tesla stock page at https://www.marketwatch.com/investing/stock/tsla and provide key financial insights"
}

print("📊 Testing MarketWatch Financial Analysis")
print("=" * 50)

result = invoke(financial_test)
print(result)

print("\n✅ MarketWatch analysis test completed!")

## 🎯 Step 6: 인터랙티브 분석 함수

In [ ]:
def analyze_website(url, question=None):
    """Convenient function for interactive website analysis"""
    if question:
        prompt = f"Visit {url} and answer this question: {question}"
    else:
        prompt = f"Please analyze the website at {url} and provide comprehensive financial insights"
    
    payload = {"prompt": prompt}
    return invoke(payload)

# 사용 예제
print("🎯 Interactive Analysis Function Ready!")
print("\nExample usage:")
print('analyze_website("https://www.marketwatch.com/investing/stock/aapl", "What is Apple\'s current stock performance?")')
print("\n✅ Interactive analysis function ready!")

## 🔄 Step 7: Main 함수

In [ ]:
if __name__ == "__main__":
    # MarketWatch 예제
    print("🎯 Production Example - Tesla Stock Analysis on MarketWatch:")
    print("=" * 60)
    
    response = invoke({
        "prompt": "Analyze the Tesla stock at https://www.marketwatch.com/investing/stock/tsla and provide detailed financial insights"
    })
    
    print(response)
    
    print("\n✅ All tests completed successfully!")
    print("\n🚀 Ready for production use with the invoke() function pattern!")
    print("\n📝 Usage Examples:")
    print('invoke({"prompt": "Analyze Tesla stock at https://www.marketwatch.com/investing/stock/tsla"})')
    print('invoke({"prompt": "Get Apple stock data from https://www.marketwatch.com/investing/stock/aapl"})')
    print('analyze_website("https://www.marketwatch.com/investing/stock/nvda", "What is NVIDIA\'s market performance?")')

## 🎭 내부에서 일어난 일

이 노트북을 실행하면 Strands Agent가 Claude Haiku model로 초기화되고 AWS Bedrock의 AgentCore 브라우저 자동화 서비스에 연결됩니다. 금융 분석 요청을 받으면 agent는 지능적으로 MarketWatch.com으로 이동하여 가격, 시가총액, P/E 비율, 거래량을 포함한 실시간 주식 데이터를 추출한 다음, 이 정보를 Claude의 AI를 통해 처리하여 포괄적인 금융 인사이트를 생성합니다.

성능 추적은 실행 속도(일반적으로 14-16초)를 측정하며, 오류 처리는 네트워크 문제나 웹사이트 차단을 우아하게 관리합니다. 전체 프로세스는 invoke() 함수 패턴을 통해 조율되며, 이는 복잡한 브라우저 자동화와 AI 처리를 관리하면서 깔끔한 인터페이스를 제공합니다.

결과는 단순한 함수 호출처럼 보이지만 실제로는 여러 AWS 서비스, AI model, 웹 스크래핑을 조정하여 신뢰할 수 있는 금융 분석을 제공합니다.

### 📝 **사용 예제:**
```python
# Direct invoke 패턴 (프로덕션)
result = invoke({"prompt": "Analyze Tesla stock at https://www.marketwatch.com/investing/stock/tsla"})

# 인터랙티브 헬퍼 함수
result = analyze_website("https://www.marketwatch.com/investing/stock/aapl", "What is Apple's current valuation?")
```
